# Imports

In [1]:
from autoboltagent.tools.low_fidelity_tool import VerboseAnalyticalTool
from autoboltagent.tools.high_fidelity_tool import VerboseFiniteElementTool

/home/matthewli125/miniconda3/envs/autoboltagent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/matthewli125/miniconda3/envs/autoboltagent/lib/python3.10/site-packages/ufl/__init__.py:250: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
from collections import deque
import numpy as np
from tqdm import tqdm

# Manual Guessing

In [3]:
joint_config = {
    "load": 60000,
    "desired_safety_factor": 3.0,
    "bolt_yield_strength": 940,
    "plate_yield_strength": 250,
    "preload": 150000,
    "pitch": 1.5,
    "plate_thickness": 10,
    "bolt_elastic_modulus": 210,
    "plate_elastic_modulus": 210
}

In [4]:
analytical_tool = VerboseAnalyticalTool(joint_configuration=joint_config)
fea_tool = VerboseFiniteElementTool(joint_configuration=joint_config)

In [ ]:
def cached_analytical_guess(past_guesses, num_bolts, bolt_diameter):
    result = analytical_tool.forward(num_bolts=num_bolts, bolt_diameter=bolt_diameter)
    past_guesses.appendleft((num_bolts, bolt_diameter, result))
    
    for row in past_guesses:
        print(row)

def cached_fea_guess(past_guesses, num_bolts, bolt_diameter):
    result = fea_tool.forward(num_bolts=num_bolts, bolt_diameter=bolt_diameter)
    past_guesses.appendleft((num_bolts, bolt_diameter, result))
    
    for row in past_guesses:
        print(row)

In [8]:
past_guesses = deque()

### Usage

Simply modify the num_bolts and bolt_diameter parameters; past guesses are saved and will be printed out as well. Note that the FEA tool generates a lot of output that can't be suppressed.

In [9]:
cached_analytical_guess(past_guesses, num_bolts=2, bolt_diameter=17)

(2, 17, {'ok': False, 'bolt_fos': 2.1834956164436536, 'bolt_diff': -0.8165043835563464, 'plate_fos': 2.125, 'plate_diff': -0.875})


In [20]:
import logging
logging.basicConfig(level=logging.CRITICAL)

cached_fea_guess(past_guesses, num_bolts=2, bolt_diameter=17)

PETSc Error --- Application was linked against both OpenMPI and MPICH based MPI libraries and will not run correctly


Info    : Reading '/tmp/tmpabrul051/model.step'...
Info    :  - Label 'Shapes/ASSEMBLY/=>[0:1:1:2]/ASSEMBLY/=>[0:1:1:3]/SOLID' (3D)
Info    :  - Label 'Shapes/ASSEMBLY/=>[0:1:1:4]/ASSEMBLY/=>[0:1:1:5]/ASSEMBLY/=>[0:1:1:6]/SOLID' (3D)
Info    :  - Label 'Shapes/ASSEMBLY/=>[0:1:1:7]/ASSEMBLY/=>[0:1:1:8]/SOLID' (3D)
Info    :  - Label 'Shapes/ASSEMBLY/=>[0:1:1:7]/ASSEMBLY/=>[0:1:1:9]/SOLID' (3D)
Info    :  - Label 'Shapes/ASSEMBLY/=>[0:1:1:3]/SOLID' (3D)
Info    :  - Label 'Shapes/ASSEMBLY/=>[0:1:1:5]/ASSEMBLY/=>[0:1:1:6]/SOLID' (3D)
Info    :  - Label 'Shapes/ASSEMBLY/=>[0:1:1:6]/SOLID' (3D)
Info    :  - Label 'Shapes/ASSEMBLY/=>[0:1:1:8]/SOLID' (3D)
Info    :  - Label 'Shapes/ASSEMBLY/=>[0:1:1:9]/SOLID' (3D)
Info    : Done reading '/tmp/tmpabrul051/model.step'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info  

# Brute Force Search

In [43]:
def brute_force_search(
    num_bolts_low, 
    num_bolts_high, 
    bolt_diameter_low, 
    bolt_diameter_high,
    step_size=0.1
):
    for num_bolts in range(num_bolts_low, num_bolts_high):
        for bolt_diameter in np.arange(bolt_diameter_low, bolt_diameter_high, step_size):
            result = analytical_tool.forward(num_bolts=num_bolts, bolt_diameter=float(bolt_diameter))

            if abs(result["bolt_diff"]) < 0.25 and abs(result["plate_diff"]) < 0.25:
                print(num_bolts, bolt_diameter,result)
            if result["ok"]:
                print("ANSWER ", num_bolts, bolt_diameter,result)

In [44]:
brute_force_search(num_bolts_low=2, num_bolts_high=40, bolt_diameter_low=3.0, bolt_diameter_high=40.0)

3 15.700000000000012 {'ok': False, 'bolt_fos': 2.7527559285221455, 'bolt_diff': -0.24724407147785454, 'plate_fos': 2.9437500000000023, 'plate_diff': -0.05624999999999769}
3 15.800000000000011 {'ok': False, 'bolt_fos': 2.7909823520750883, 'bolt_diff': -0.20901764792491173, 'plate_fos': 2.962500000000002, 'plate_diff': -0.03749999999999787}
3 15.900000000000011 {'ok': False, 'bolt_fos': 2.829900224552613, 'bolt_diff': -0.1700997754473872, 'plate_fos': 2.9812500000000024, 'plate_diff': -0.018749999999997602}
3 16.00000000000001 {'ok': False, 'bolt_fos': 2.8690875598645036, 'bolt_diff': -0.1309124401354964, 'plate_fos': 3.000000000000002, 'plate_diff': 2.220446049250313e-15}
3 16.100000000000012 {'ok': True, 'bolt_fos': 2.9085443580107597, 'bolt_diff': -0.09145564198924028, 'plate_fos': 3.018750000000002, 'plate_diff': 0.018750000000002043}
ANSWER  3 16.100000000000012 {'ok': True, 'bolt_fos': 2.9085443580107597, 'bolt_diff': -0.09145564198924028, 'plate_fos': 3.018750000000002, 'plate_dif